In [1]:
import kplr
import copy
import emcee
import corner

from autograd import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import minimize

from celerite import plot_setup, terms